<a href="https://colab.research.google.com/github/ianwolf99/Generative-AI/blob/main/Chatbot_websites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install langchain
# !pip install faiss-cpu
# !pip install openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR-OPENAI_API_KEY"

In [ ]:
urls = [
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'
]

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
loaders = UnstructuredURLLoader(urls=urls)
data = loaders.load()

In [ ]:
data

[Document(page_content='All\n\nIntegrations\n\nEngineering\n\nResearch\n\nLaunch\n\nMethodology\n\nMore\n\nLink 1\n\nIntegrations\n\nEngineering\n\nResearch\n\nLaunch\n\nMethodology\n\nResearch\n\nby\n\nThe MosaicML NLP Team\n\non\n\nMay 5, 2023\n\nShare\n\nIntroducing MPT-7B: A New Standard for Open-Source, Commercially Usable LLMs\n\nIntroducing MPT-7B, the latest entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k. Starting today, you can train, finetune, and deploy your own private MPT models, either starting from one of our checkpoints or training from scratch. For inspiration, we are also releasing three finetuned models in addition to the base MPT-7B: MPT-7B-Instruct, MPT-7B-Chat, and MPT-7B-StoryWriter-65k+, the last of which 

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)


docs = text_splitter.split_documents(data)

In [ ]:
docs

[Document(page_content='All\n\nIntegrations\n\nEngineering\n\nResearch\n\nLaunch\n\nMethodology\n\nMore\n\nLink 1\n\nIntegrations\n\nEngineering\n\nResearch\n\nLaunch\n\nMethodology\n\nResearch\n\nby\n\nThe MosaicML NLP Team\n\non\n\nMay 5, 2023\n\nShare\n\nIntroducing MPT-7B: A New Standard for Open-Source, Commercially Usable LLMs\n\nIntroducing MPT-7B, the latest entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k. Starting today, you can train, finetune, and deploy your own private MPT models, either starting from one of our checkpoints or training from scratch. For inspiration, we are also releasing three finetuned models in addition to the base MPT-7B: MPT-7B-Instruct, MPT-7B-Chat, and MPT-7B-StoryWriter-65k+, the last of which 

In [ ]:
len(docs)

62

In [ ]:
import pickle
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='2022-12-01', openai_api_base=None, openai_api_type=None, embedding_ctx_length=8191, openai_api_key=None, openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6)

In [ ]:
# vectorStore_openAI = FAISS.from_documents(docs, embeddings)

# with open("faiss_store_openai.pkl", "wb") as f:
#     pickle.dump(vectorStore_openAI, f)

In [ ]:
with open("faiss_store_openai.pkl", "rb") as f:
    VectorStore = pickle.load(f)

In [ ]:
VectorStore

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm=OpenAI(temperature=0, model_name='')

In [ ]:
llm

OpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.0, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, allowed_special=set(), disallowed_special='all')

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=VectorStore.as_retriever())

In [ ]:

chain({"question": "How big is stableLM?"}, return_only_outputs=True)


{'answer': ' StableLM is available in 3 billion and 7 billion parameters, with 15 billion to 65 billion parameter models to follow.\n',
 'sources': 'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models'}

In [ ]:
chain({"question": "How good is Vicuna?"}, return_only_outputs=True)

{'answer': ' Vicuna is capable of generating detailed and well-structured answers with a quality on par with ChatGPT, outperforming other models like LLaMA and Stanford Alpaca in more than 90% of cases. However, it has certain limitations such as not being good at tasks involving reasoning or mathematics.\n',
 'sources': '\nhttps://lmsys.org/blog/2023-03-30-vicuna/'}

In [ ]:
chain({"question": "Which MPT-7B model is the bast one?"}, return_only_outputs=True)

{'answer': ' The best MPT-7B model is MPT-7B-Instruct, which was trained on 1 trillion tokens.\n',
 'sources': 'https://www.mosaicml.com/blog/mpt-7b'}